In [1]:
from ase import Atoms
from ase.build import fcc100, add_vacuum
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.data import atomic_numbers
from ase.ga.startgenerator import StartGenerator
from ase.ga.utilities import closest_distances_generator
from ase.ga.utilities import get_all_atom_types
from ase.io import read
from ase.optimize.sciopt import SciPyFminCG
from ase.visualize import view

import numpy as np


In [2]:
surface_size = (6, 6, 3)
slab = fcc100('Au', surface_size, periodic=True)
add_vacuum(slab, 10)
slab.set_constraint(FixAtoms(mask=len(slab) * [True]))

# define the volume in which the adsorbed cluster is optimized
# the volume is defined by a corner position (p0)
# and three spanning vectors (v1, v2, v3)
pos = slab.get_positions()
cell = slab.get_cell()
print(cell[0,0])
p0 = np.array([cell[0, 0]/10*3, cell[1, 1]/10*3, max(pos[:, 2]) + 2.])
v1 = cell[0, :] * 0.4
v2 = cell[1, :] * 0.4
v3 = cell[2, :] * .3

# Define the composition of the atoms to optimize
atom_numbers = 12 * [atomic_numbers['Pt']]

# define the closest distance two atoms of a given species can be to each other
unique_atom_types = get_all_atom_types(slab, atom_numbers)
blmin = closest_distances_generator(atom_numbers=unique_atom_types,
                                    ratio_of_covalent_radii=0.7)

# create the starting population
sg = StartGenerator(slab, atom_numbers, blmin,
                    box_to_place_in=[p0, [v1, v2, v3]])
# view(slab)


17.309974003446687


In [3]:
atoms = sg.get_new_candidate()
atoms.set_calculator(EMT())
traj_name = 'images/pt12_au100_emt.traj'
dyn = SciPyFminCG(atoms, logfile='-', trajectory=traj_name)
dyn.run(fmax=0.25)

             Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
SciPyFminCG:    0 00:14:36       57.022629*      28.4709
SciPyFminCG:    1 00:14:36       42.863694*       5.4934
SciPyFminCG:    2 00:14:36       40.860503*       3.6512
SciPyFminCG:    3 00:14:36       39.963107*       3.0119
SciPyFminCG:    4 00:14:36       39.497727*       2.2062
SciPyFminCG:    5 00:14:36       39.252795*       1.9846
SciPyFminCG:    6 00:14:37       39.047227*       2.0199
SciPyFminCG:    7 00:14:37       38.701125*       1.8125
SciPyFminCG:    8 00:14:37       38.481046*       0.9924
SciPyFminCG:    9 00:14:37       38.338837*       1.3292
SciPyFminCG:   10 00:14:37       38.121889*       1.2061
SciPyFminCG:   11 00:14:37       38.042185*       1.1895
SciPyFminCG:   12 00:14:37       37.890389*       0.9604
SciPyFminCG:   13 00:14:37       37.749982*       1.1256
SciPyFminCG:   14 00:14:37       37.628636*       1.5393
SciPyFminCG:   15 00:14:37       37.1599

In [4]:
traj = read(traj_name, index=':')
view(traj, viewer='ngl')